In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from cycler import cycler
import tensorflow as tf
import seaborn as sns
import pandas
import datetime
from imdb import IMDb
from mediawikiapi import MediaWikiAPI
from tqdm import tqdm

In [3]:
color_cycle_wong = ['#000000','#E69F00','#CC79A7','#56B4E9','#009E73','#0072B2','#F0E442','#D55E00']
Six_color_cycle = ["#5790FC", "#F89C20", "#E42536", "#964A8B", "#9C9CA1", "#7A21DD"]
Eight_color_cycle = ["#1845FB", "#FF5E02", "#C91F16", "#C849A9", "#ADAD7D", "#86C8DD", "#578DFF", "#656364"]
Ten_color_cycle = ["#3F90DA", "#FFA90E", "#BD1F01", "#94A4A2", "#832DB6", "#A96B59", "#E76300", "#B9AC70", "#717581", "#92DADD"]
mpl.rcParams['axes.prop_cycle'] = cycler(color=Six_color_cycle)

In [4]:
movies = pandas.read_csv('./Misc_Files/movies.csv')
#votes = pandas.read_csv('./Misc_Files/results.csv')

In [5]:
mediawikiapi = MediaWikiAPI()
lists_of_films_page = mediawikiapi.page("Lists of films")
subpage_names = [subpage for subpage in lists_of_films_page.links if 'List of films:' in subpage]
subpage_names

['List of films: A',
 'List of films: B',
 'List of films: C',
 'List of films: D',
 'List of films: E',
 'List of films: F',
 'List of films: G',
 'List of films: H',
 'List of films: I',
 'List of films: J–K',
 'List of films: L',
 'List of films: M',
 'List of films: N–O',
 'List of films: P',
 'List of films: Q–R',
 'List of films: S',
 'List of films: T',
 'List of films: U–W',
 'List of films: X–Z',
 'List of films: numbers']

In [6]:
lists_of_films = []
for subpage in subpage_names:
    try:
        page = mediawikiapi.page(subpage)
        lists_of_films.extend(page.links)
    except:
        e = sys.exc_info()[0]
        print(f"Error: {e} on {subpage}")
lists_of_films = np.asarray(lists_of_films)
print(np.shape(lists_of_films))

Error: <class 'mediawikiapi.exceptions.PageError'> on List of films: G
Error: <class 'mediawikiapi.exceptions.PageError'> on List of films: H
Error: <class 'mediawikiapi.exceptions.PageError'> on List of films: L
(44768,)


In [7]:
def get_movie_info(movie_name):
    print(movie_name)
    search_movie = ia.search_movie(movie_name)
    if search_movie:
        if search_movie[0]['kind'] in ['movie','video movie']:
            imdb_movie = ia.get_movie(search_movie[0].movieID)
        else:
            i=1
            try:
                while search_movie[i]['kind'] != 'movie':
                    i+=1
                imdb_movie = ia.get_movie(search_movie[i].movieID)
            except:
                return 0.0
        print(imdb_movie['title'], 'found.')
        return imdb_movie
    else:
        return 0.0

In [8]:
# create an instance of the IMDb class
ia = IMDb()

In [9]:
perp=get_movie_info('Space Cop (2016)')
perp['genre']

Space Cop (2016)
Space Cop found.


['Action', 'Comedy', 'Sci-Fi']

In [10]:
tmp = ia.search_movie('Space Cop (2016)')[0]
burp = ia.get_movie(tmp.movieID)
print(burp.current_info)
print(burp['genre'])
print(burp.keys())
print(burp['kind'])

['main', 'plot', 'synopsis']
['Action', 'Comedy', 'Sci-Fi']
['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'plot outline', 'languages', 'title', 'year', 'kind', 'directors', 'writers', 'producers', 'composers', 'cinematographers', 'editors', 'sound department', 'visual effects', 'stunts', 'akas', 'writer', 'director', 'production companies', 'plot', 'canonical title', 'long imdb title', 'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title', 'full-size cover url']
video movie


DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']


np.shape(train_examples)

np.shape(train_labels)

In [11]:
interesting_parameters = ['title','genres','year','languages','votes','runtimes','rating']

In [12]:
all(ip in burp.keys() for ip in interesting_parameters)

True

In [13]:
def check_if_good_movie(imdb_movie,interesting_parameters):
    if isinstance(imdb_movie,float):
        return False
    elif any(ip not in imdb_movie.keys() for ip in interesting_parameters):
        return False
    else:
        if 'rating' in imdb_movie.keys():
            if imdb_movie['rating'] == 0.:
                 return False
        else:
            return False
        if 'runtimes' in imdb_movie.keys():
            if imdb_movie['runtimes'][0] == 0.:
                 return False
        else:
            return False
        if 'votes' in imdb_movie.keys():
            if imdb_movie['votes'] < 500.:
                 return False
        else:
            return False
    return True
def split_movie_name(movie_name):
    if '(film)' in movie_name:
        movie_name = movie_name.split('(film)')[0]
    elif ' film' in movie_name:
        movie_name = movie_name.split(' film)')[0] + ')'
    return movie_name

In [14]:
save_data_file = './Misc_Files/imdb_movie_selection_v1.npy'
add_data = False
save_slice = 100
if add_data:
    if os.path.isfile(save_data_file):
        movie_selection_text = np.load(save_data_file,allow_pickle=True)
        movie_selection_text = pandas.DataFrame(loaded_data,columns=interesting_parameters)
    else:
        movie_selection_text = pandas.DataFrame(data=None, index=None, columns=interesting_parameters, dtype=None, copy=False)

    num_movies = 1000
    rand_movies = [split_movie_name(mn) for mn in np.random.choice(lists_of_films,num_movies,replace=False)]
    for i,movie_name in enumerate(tqdm(rand_movies)):
        imdb_movie = get_movie_info(movie_name)
        good_movie = check_if_good_movie(imdb_movie)
        while not good_movie:
            #If something went wrong with the movie, pick a different one
            rand_movies[i] = split_movie_name(np.random.choice(lists_of_films,1,replace=False)[0])
            imdb_movie = get_movie_info(rand_movies[i])
            good_movie = check_if_good_movie(imdb_movie)
        row_insert = {}
        for par in interesting_parameters:
            if par in imdb_movie.keys():
                if par in ['genres','director','languages']:
                    row_insert[par] = str(imdb_movie[par][0])
                elif par in ['runtimes']:
                    row_insert[par] = float(imdb_movie[par][0])
                else:
                    row_insert[par] = imdb_movie[par]
            else:
                if par in ['title','genres','director','languages']:
                    row_insert[par] = 'N/A'
                else:
                    row_insert[par] = 0.
        if i == 0 and not os.path.isfile(save_data_file):
            movie_selection_text = pandas.DataFrame([row_insert])
        else:
            movie_selection_text = movie_selection_text.append(row_insert,ignore_index=True)
        if i%save_slice == 0 and i != 0:
            print('')
            print(f'Saving! i={i}')
            print('')
            np.save(save_data_file,movie_selection_text.to_numpy())
    np.save(save_data_file,movie_selection_text.to_numpy())
else:
    print('loading data...')
    loaded_data = np.load(save_data_file,allow_pickle=True)
    movie_selection_text = pandas.DataFrame(loaded_data,columns=interesting_parameters)

loading data...


In [15]:
movie_selection_text

,title,genres,year,languages,votes,runtimes,rating
0,Tuscan Wedding,Comedy,2014,English,1790,104,5.2
1,The Four of the Apocalypse...,Fantasy,1975,Italian,2485,104,6.4
2,Knuckleball!,Documentary,2012,English,1686,93,7.1
3,The Farewell,Comedy,2019,Mandarin,56213,100,7.6
4,Johnny in the Clouds,Drama,1945,English,1441,109,7.3
...,...,...,...,...,...,...,...
5697,Summer Lovers,Comedy,1982,English,3971,98,5.7
5698,Matchstick Men,Comedy,2003,English,126764,116,7.3
5699,Sands of Iwo Jima,Action,1949,English,10065,100,7.1
5700,The Nutcracker,Family,1993,English,1968,92,5.8


### Convert `movie_selection_text` to numerical tensor
* Convert strings to floats
* Take out non-used column (title)
* Take out solution column (rating) and make appropriate shape
* Normalize columns
* Convert Nx5 matrix to N_groups x 5 x (N_movies/N_groups)

In [16]:
def normalize_cols(df):
    normed_df = df.copy()
    for key in normed_df.keys():
        normed_df[key] = normed_df[key]  / normed_df[key].abs().max()
    return normed_df

In [17]:
genre_map = {genre:float(i) for i,genre in enumerate(movie_selection_text['genres'].value_counts().keys())}
language_map = {lang:float(i) for i,lang in enumerate(movie_selection_text['languages'].value_counts().keys())}

In [37]:
movie_num_map = movie_selection_text.copy()
movie_num_map = movie_num_map.replace(to_replace=genre_map)
movie_num_map = movie_num_map.replace(to_replace=language_map)
movie_num_map.pop('title')

target = movie_num_map.pop('rating')
movie_num_map["runtimes"] = pandas.to_numeric(movie_num_map["runtimes"])
movie_num_map = normalize_cols(movie_num_map)

In [38]:
n_groups = 5
trimmed_movies = (np.shape(movie_num_map)[0]%n_groups)
movie_num_map = movie_num_map.to_numpy()
print(f'trimmed {trimmed_movies} movies.')
movie_num_map = movie_num_map[:-trimmed_movies].reshape(-1,n_groups,
                                                        np.shape(movie_num_map)[1])
target = target.to_numpy()
target = target[:-trimmed_movies].reshape(n_groups,-1)
winners = np.asarray([np.where(target[:,t]==np.max(target[:,t]))[0][0] for t in range(np.shape(target)[1])]).astype(np.int32)
winners = winners.astype(np.int32)
print(np.shape(target))
print(np.shape(winners))
print(np.shape(movie_num_map))
#movie_num_map.shape()

trimmed 2 movies.
(5, 1140)
(1140,)
(1140, 5, 5)


In [39]:
winners.shape

(1140,)

# TensorFlow Start

In [40]:
#movie_num_map = tf.convert_to_tensor(movie_num_map)
movie_num_map = tf.data.Dataset.from_tensor_slices((movie_num_map, winners))
movie_num_map = movie_num_map.shuffle(5000).batch(32)

#movie_num_map = movie_num_map.shuffle(5000).batch(10)

#movie_num_map[0]

In [41]:
movie_num_map.element_spec

(TensorSpec(shape=(None, 5, 5), dtype=tf.float64, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [42]:
for asdf in movie_num_map.as_numpy_iterator():
    print(np.shape(asdf[0]),asdf[1])

(32, 5, 5) [1 3 3 0 3 3 2 3 3 1 3 3 3 0 0 0 4 3 2 1 0 4 1 2 3 2 3 2 3 2 3 2]
(32, 5, 5) [2 4 0 2 3 4 0 3 1 1 1 1 2 1 0 4 1 4 0 2 0 1 1 1 1 4 0 2 3 0 0 4]
(32, 5, 5) [3 2 0 3 0 1 4 3 2 1 0 1 3 0 0 0 0 2 4 2 3 4 4 4 4 1 1 2 4 1 1 0]
(32, 5, 5) [0 0 3 0 0 2 1 2 3 4 1 0 3 2 0 1 2 0 4 1 3 0 0 3 2 4 1 2 3 4 0 1]
(32, 5, 5) [0 0 1 2 2 4 3 0 0 2 3 2 1 1 2 4 3 1 4 3 0 1 0 1 3 1 3 3 3 2 2 3]
(32, 5, 5) [4 0 2 3 1 2 1 3 1 4 1 0 2 2 1 0 0 4 2 4 2 3 0 1 2 1 4 2 1 4 1 2]
(32, 5, 5) [1 0 3 0 1 1 3 1 1 2 0 3 3 0 4 1 1 4 4 4 1 3 4 1 2 3 0 4 2 1 4 1]
(32, 5, 5) [2 4 1 3 0 4 0 2 4 3 1 3 1 1 0 1 2 3 3 1 2 3 0 2 0 1 1 2 4 1 3 2]
(32, 5, 5) [3 4 3 0 2 4 2 4 3 2 2 3 2 1 1 2 4 3 0 0 4 0 4 3 1 1 3 2 3 1 4 0]
(32, 5, 5) [4 0 2 0 2 2 2 4 1 2 2 0 2 1 3 0 0 1 0 1 0 1 1 0 1 4 0 2 0 0 1 0]
(32, 5, 5) [1 0 0 0 2 3 1 0 4 1 0 2 0 3 0 3 4 3 1 3 3 4 1 3 0 0 4 1 0 1 1 3]
(32, 5, 5) [1 1 1 2 3 0 0 1 3 3 4 1 2 2 2 4 4 0 4 0 0 4 4 0 4 1 0 4 2 2 3 1]
(32, 5, 5) [1 4 2 1 2 4 1 2 0 0 4 0 2 3 0 4 0 4 0 1 3 4 0 2 1 3 4 2 3 1 2 2]

In [45]:
model = tf.keras.Sequential([tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128),
                             tf.keras.layers.Dense(50),
                             tf.keras.layers.Dense(10)])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#model.fit(movie_num_map.repeat(), epochs=2, steps_per_epoch=20)
history = model.fit(movie_num_map.repeat(), epochs=100, verbose=True, steps_per_epoch=20)
model.summary()

Epoch 1/100
20/20 [==============================] - 0s 1ms/step - loss: 2.0416 - accuracy: 0.1641
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6503 - accuracy: 0.2182
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6212 - accuracy: 0.2406
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 1.6207 - accuracy: 0.2006
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6160 - accuracy: 0.2313
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 1.6124 - accuracy: 0.2373
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6225 - accuracy: 0.2359
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 1.6082 - accuracy: 0.2484
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6136 - accuracy: 0.2357
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6135 - accuracy: 0.2203
Epoch 11/

20/20 [==============================] - 0s 1ms/step - loss: 1.5743 - accuracy: 0.2691
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6051 - accuracy: 0.2359
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 1.5736 - accuracy: 0.2755
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 1.5925 - accuracy: 0.2531
Epoch 87/100
20/20 [==============================] - 0s 2ms/step - loss: 1.5879 - accuracy: 0.2389
Epoch 88/100
20/20 [==============================] - 0s 2ms/step - loss: 1.5770 - accuracy: 0.2703
Epoch 89/100
20/20 [==============================] - 0s 2ms/step - loss: 1.6070 - accuracy: 0.2420
Epoch 90/100
20/20 [==============================] - 0s 2ms/step - loss: 1.5822 - accuracy: 0.2659
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 1.5789 - accuracy: 0.2828
Epoch 92/100
20/20 [==============================] - 0s 2ms/step - loss: 1.5853 - accuracy: 0.2500
Epoch 93/100


In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(10,5,1),
                             #tf.keras.layers.Dense(10, activation='relu'),
                             tf.keras.layers.Dense(10, activation = tf.keras.activations.softmax)])
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
#BATCH_SIZE = 32
#train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
#test_dataset = test_dataset.cache().batch(BATCH_SIZE)
#model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))
history = model.fit(movie_num_map, epochs=100, verbose=True)

# Tensor Flow Examples

```
celsius_q    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit_a = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)
l0 = tf.keras.layers.Dense(units=1, input_shape=[1]) 
model = tf.keras.Sequential([l0])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1))
model.summary()
```

```
tf.keras.utils.plot_model(model, show_shapes=True)
```

```
history = model.fit(celsius_q, fahrenheit_a, epochs=500, verbose=False)
model.predict([100.0])
```

## Transfer Learning